In [1]:
import pandas as pd
import numpy as np 
import os 
import nltk 
from tqdm import tqdm
from multiprocessing import Pool
print("Hello")

Hello


In [9]:
df=pd.read_csv("new_8_train.csv")
print(df.shape)
df.head()

(404287, 39)


,question1,question2,is_duplicate,q1,q2,q1_ques_count,q2_ques_count,questions_count_diff,q1_norm_intersection,q2_norm_intersection,...,q2ADV,q2DET,q2NOUN,q2PRON,q2PRT,q2VERB,q2ADJ,q2NUM,q2CONJ,q2X
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"['what', 'is', 'the', 'step', 'by', 'step', 'g...","['what', 'is', 'the', 'step', 'by', 'step', 'g...",1,1,0,0.933333,1.000000,...,1.0,1.0,4.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"['what', 'is', 'the', 'story', 'of', 'kohinoor...","['what', 'would', 'happen', 'if', 'the', 'indi...",1,1,0,0.727273,0.562500,...,1.0,2.0,3.0,1.0,0.0,3.0,2.0,0.0,0.0,0.0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"['how', 'can', 'i', 'increase', 'the', 'speed'...","['how', 'can', 'internet', 'speed', 'be', 'inc...",1,1,0,0.333333,0.454545,...,1.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"['why', 'am', 'i', 'mentally', 'very', 'lonely...","['find', 'the', 'remainder', 'when', '[', 'mat...",2,1,-1,0.153846,0.052632,...,1.0,1.0,6.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"['which', 'one', 'dissolve', 'in', 'water', 'q...","['which', 'fish', 'would', 'survive', 'in', 's...",1,1,0,0.312500,0.625000,...,0.0,1.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [4]:
def getPosIou(q1,q2):
    q1=eval(q1)
    q1tags=nltk.pos_tag(q1,tagset="universal")
    q2=eval(q2)
    q2tags=nltk.pos_tag(q2,tagset="universal")
    df1=pd.DataFrame(q1tags,columns=['tok','tag'])
    df2=pd.DataFrame(q2tags,columns=['tok','tag'])
    dict1={}
    for i in df1.tag.unique():
        dict1[i]=list(df1.tok[df1['tag']==i])
    dict2={}
    for i in df2.tag.unique():
        dict2[i]=list(df2.tok[df2['tag']==i])
    
    output={}
    for i in dict1.keys():
        if i not in dict2.keys():
            continue
        else:
            set1=set(dict1[i])
            set2=set(dict2[i])
            inter=set1.intersection(set2)
            uni=set1.union(set2)
            output[i+"IOU"]=round(len(inter)/len(uni),2)
    return output

In [5]:
output=[]
for i in tqdm(df.itertuples()):
    q1=i.q1
    q2=i.q2
    out=getPosIou(q1,q2)
    output.append(out)

404287it [1:17:14, 87.24it/s] 


In [10]:
pos_df=pd.DataFrame(output)
pos_df.columns

Index(['PRONIOU', 'VERBIOU', 'DETIOU', 'NOUNIOU', 'ADPIOU', 'ADVIOU', 'PRTIOU',
       '.IOU', 'ADJIOU', 'CONJIOU', 'NUMIOU', 'XIOU'],
      dtype='object')

In [11]:
pos_df.head(10)

,PRONIOU,VERBIOU,DETIOU,NOUNIOU,ADPIOU,ADVIOU,PRTIOU,.IOU,ADJIOU,CONJIOU,NUMIOU,XIOU
0,1.0,1.00,1.00,0.75,1.0,1.0,1.0,1.00,NaN,NaN,NaN,NaN
1,1.0,0.00,1.00,0.50,0.0,NaN,NaN,1.00,0.5,NaN,NaN,NaN
2,NaN,0.12,NaN,0.25,0.0,1.0,NaN,1.00,NaN,NaN,NaN,NaN
3,NaN,0.00,NaN,NaN,NaN,0.0,NaN,0.33,NaN,NaN,NaN,NaN
4,NaN,NaN,1.00,0.22,0.5,NaN,NaN,0.50,NaN,NaN,NaN,NaN
5,1.0,0.50,0.33,0.38,0.5,NaN,NaN,0.17,0.0,1.0,NaN,NaN
6,NaN,0.00,NaN,NaN,0.0,NaN,NaN,1.00,NaN,NaN,NaN,NaN
7,NaN,0.40,1.00,1.00,NaN,NaN,NaN,1.00,0.0,NaN,NaN,NaN
8,1.0,1.00,NaN,NaN,1.0,1.0,NaN,0.33,NaN,NaN,NaN,NaN
9,NaN,0.25,NaN,0.29,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN


In [12]:
pos_df.fillna(999,inplace=True)
pos_df.head(10)

,PRONIOU,VERBIOU,DETIOU,NOUNIOU,ADPIOU,ADVIOU,PRTIOU,.IOU,ADJIOU,CONJIOU,NUMIOU,XIOU
0,1.0,1.00,1.00,0.75,1.0,1.0,1.0,1.00,999.0,999.0,999.0,999.0
1,1.0,0.00,1.00,0.50,0.0,999.0,999.0,1.00,0.5,999.0,999.0,999.0
2,999.0,0.12,999.00,0.25,0.0,1.0,999.0,1.00,999.0,999.0,999.0,999.0
3,999.0,0.00,999.00,999.00,999.0,0.0,999.0,0.33,999.0,999.0,999.0,999.0
4,999.0,999.00,1.00,0.22,0.5,999.0,999.0,0.50,999.0,999.0,999.0,999.0
5,1.0,0.50,0.33,0.38,0.5,999.0,999.0,0.17,0.0,1.0,999.0,999.0
6,999.0,0.00,999.00,999.00,0.0,999.0,999.0,1.00,999.0,999.0,999.0,999.0
7,999.0,0.40,1.00,1.00,999.0,999.0,999.0,1.00,0.0,999.0,999.0,999.0
8,1.0,1.00,999.00,999.00,1.0,1.0,999.0,0.33,999.0,999.0,999.0,999.0
9,999.0,0.25,999.00,0.29,999.0,999.0,999.0,0.25,999.0,999.0,999.0,999.0


In [13]:
pos_df.to_csv("posiou.csv",index=None)